

---

# **Chapter 7: Dynamic Extensions and Aggregation**

## **Opening Context**
Inheritance is static—defined at compile time, fixed for the object's lifetime. Yet modern applications require behavior that evolves dynamically: UI components that gain scrolling borders at runtime, file systems where folders contain other folders, shapes that render via different drawing APIs without code duplication, and games that render thousands of similar trees without exhausting memory. This chapter addresses four patterns that achieve flexibility through composition rather than inheritance. The Decorator enables runtime behavior stacking; Composite handles recursive tree structures; Bridge separates abstraction from implementation; and Flyweight enables massive-scale object sharing.

---

## **7.1 Decorator Pattern: Adding Responsibilities Dynamically**

### **Intent**
*Attach additional responsibilities to an object dynamically. Decorators provide a flexible alternative to subclassing for extending functionality.*

### **The Problem**
Consider a coffee shop application calculating beverage costs. Using inheritance to add condiments creates a **combinatorial explosion**:

```typescript
// ❌ INHERITANCE HELL: Class explosion
class Coffee { cost(): number { return 2; } }
class CoffeeWithMilk extends Coffee { cost(): number { return 2.5; } }
class CoffeeWithSugar extends Coffee { cost(): number { return 2.3; } }
class CoffeeWithMilkAndSugar extends Coffee { cost(): number { return 2.8; } }
class CoffeeWithMilkAndSugarAndWhip extends Coffee { /* ... */ }
// 2^n classes for n condiments!
```

**Problems**:
- **Static binding**: Condiment combination decided at compile time.
- **Code duplication**: `Milk` logic duplicated in `CoffeeWithMilk` and `CoffeeWithMilkAndSugar`.
- **Inflexibility**: Cannot add/remove condiments at runtime.

### **The Solution: Recursive Composition**

The Decorator pattern uses composition to wrap objects, creating a **stack of responsibilities** that can be combined dynamically.

```typescript
/**
 * COMPONENT INTERFACE
 * Defines the interface for objects that can have responsibilities
 * added to them dynamically.
 */
interface Coffee {
    cost(): number;
    getDescription(): string;
}

/**
 * CONCRETE COMPONENT
 * The base object to which additional responsibilities can be attached.
 */
class SimpleCoffee implements Coffee {
    cost(): number {
        return 2.0; // Base price
    }
    
    getDescription(): string {
        return "Simple coffee";
    }
}

/**
 * DECORATOR (Base Class)
 * Maintains a reference to a Component and implements the Component interface.
 * Delegates operations to the wrapped component, allowing subclasses to
 * add behavior before/after delegation.
 */
abstract class CoffeeDecorator implements Coffee {
    // The component being wrapped (could be ConcreteComponent or another Decorator)
    protected decoratedCoffee: Coffee;
    
    constructor(coffee: Coffee) {
        this.decoratedCoffee = coffee;
    }
    
    // Default implementation delegates to wrapped component
    cost(): number {
        return this.decoratedCoffee.cost();
    }
    
    getDescription(): string {
        return this.decoratedCoffee.getDescription();
    }
}

/**
 * CONCRETE DECORATORS
 * Add specific responsibilities by extending the Decorator.
 * Each decorator adds its own behavior then delegates to the wrapped object.
 */
class MilkDecorator extends CoffeeDecorator {
    constructor(coffee: Coffee) {
        super(coffee);
    }
    
    cost(): number {
        // Add milk cost, then delegate to wrapped component for its cost
        return this.decoratedCoffee.cost() + 0.5;
    }
    
    getDescription(): string {
        return this.decoratedCoffee.getDescription() + ", milk";
    }
}

class SugarDecorator extends CoffeeDecorator {
    cost(): number {
        return this.decoratedCoffee.cost() + 0.3;
    }
    
    getDescription(): string {
        return this.decoratedCoffee.getDescription() + ", sugar";
    }
}

class WhipDecorator extends CoffeeDecorator {
    cost(): number {
        return this.decoratedCoffee.cost() + 0.7;
    }
    
    getDescription(): string {
        return this.decoratedCoffee.getDescription() + ", whip";
    }
}

// Special decorator with additional behavior
class VanillaDecorator extends CoffeeDecorator {
    private vanillaShotCount: number;
    
    constructor(coffee: Coffee, shots: number = 1) {
        super(coffee);
        this.vanillaShotCount = shots;
    }
    
    cost(): number {
        return this.decoratedCoffee.cost() + (0.4 * this.vanillaShotCount);
    }
    
    getDescription(): string {
        return `${this.decoratedCoffee.getDescription()}, ${this.vanillaShotCount}x vanilla`;
    }
}

// Usage: Dynamic composition at runtime
const order: Coffee = new VanillaDecorator(
    new WhipDecorator(
        new MilkDecorator(
            new SugarDecorator(
                new SimpleCoffee()
            )
        )
    ),
    2 // 2 shots of vanilla
);

console.log(order.getDescription()); 
// Output: Simple coffee, sugar, milk, whip, 2x vanilla
console.log(`Total: $${order.cost()}`); 
// Output: Total: $4.3 (2.0 + 0.3 + 0.5 + 0.7 + 0.8)
```

**Code Explanation**:

- **Component** (`Coffee`): Defines the interface for concrete components and decorators.
- **ConcreteComponent** (`SimpleCoffee`): The base object being wrapped.
- **Decorator** (`CoffeeDecorator`): Maintains a reference to a `Coffee` (the wrapped object) and implements the same interface. This is crucial—the decorator *is-a* Coffee and *has-a* Coffee, enabling recursive wrapping.
- **ConcreteDecorators** (`MilkDecorator`, etc.): Each adds specific behavior (cost, description) then delegates to `decoratedCoffee` for the rest. This creates a **chain of responsibility**.

**Structure Visualization**:
```
VanillaDecorator (cost: +0.8, desc: +", 2x vanilla")
    ↓ decoratedCoffee
WhipDecorator (cost: +0.7, desc: +", whip")
    ↓ decoratedCoffee
MilkDecorator (cost: +0.5, desc: +", milk")
    ↓ decoratedCoffee
SugarDecorator (cost: +0.3, desc: +", sugar")
    ↓ decoratedCoffee
SimpleCoffee (cost: 2.0, desc: "Simple coffee")
```

**Call Flow**:
1. `order.cost()` calls `VanillaDecorator.cost()`
2. Which calls `WhipDecorator.cost()` via `decoratedCoffee.cost()`
3. Which calls `MilkDecorator.cost()`
4. ...until `SimpleCoffee.cost()` returns 2.0
5. Each decorator adds its cost on the return trip up the stack.

### **Decorator vs. Inheritance**
| Aspect | Inheritance | Decorator |
|--------|-------------|-----------|
| **Binding Time** | Compile time | Runtime |
| **Flexibility** | Fixed hierarchy | Can add/remove dynamically |
| **Combination** | Requires new class for each combination | Combines existing decorators |
| **Transparency** | Client knows concrete subclass | Client sees only Component interface |

### **Real-World Application: I/O Streams**
The Java/Node.js stream APIs are classic Decorator implementations:

```typescript
// Node.js streams as Decorator pattern
import { createReadStream } from 'fs';
import { createGzip } from 'zlib';
import { createCipheriv } from 'crypto';

// Simple component: File stream
const fileStream = createReadStream('data.txt');

// Decorator 1: Add compression
const gzipStream = createGzip(); // Decorator
fileStream.pipe(gzipStream);

// Decorator 2: Add encryption (decorating the compressed stream)
const cipher = createCipheriv('aes-256-cbc', key, iv);
gzipStream.pipe(cipher);

// Result: File → Compressed → Encrypted
// Each stream wraps the previous, adding responsibility
```

---

## **7.2 Composite Pattern: Treating Objects as Trees**

### **Intent**
*Compose objects into tree structures to represent part-whole hierarchies. Composite lets clients treat individual objects and compositions of objects uniformly.*

### **The Problem**
File systems, UI component trees, and organizational hierarchies contain both **individual items** (files, leaf components) and **containers** (folders, composite components) that hold other items. Clients must manipulate these through different interfaces, creating complex conditional logic:

```typescript
// ❌ WITHOUT COMPOSITE: Client must distinguish between File and Folder
function getSize(item: File | Folder): number {
    if (item instanceof File) {
        return item.size;
    } else {
        // Folder: sum all children
        return item.children.reduce((sum, child) => {
            if (child instanceof File) {
                return sum + child.size;
            } else {
                // Recursion needed, but types are messy
                return sum + getSize(child);
            }
        }, 0);
    }
}
```

### **The Solution: Uniform Interface**

The Composite pattern defines a common interface (`Component`) that both individual objects (`Leaf`) and compositions (`Composite`) implement. The Composite stores child components and delegates operations to them.

```typescript
/**
 * COMPONENT INTERFACE
 * Declares the interface for objects in the composition.
 * Both Leaf and Composite implement this.
 */
interface FileSystemComponent {
    getName(): string;
    getSize(): number;  // Key: same interface for file and folder
    print(indent?: string): void;
}

/**
 * LEAF
 * Represents individual objects (files) with no children.
 */
class File implements FileSystemComponent {
    constructor(
        private name: string,
        private size: number
    ) {}
    
    getName(): string {
        return this.name;
    }
    
    getSize(): number {
        return this.size;
    }
    
    print(indent: string = ""): void {
        console.log(`${indent}📄 ${this.name} (${this.size} bytes)`);
    }
}

/**
 * COMPOSITE
 * Represents containers (folders) that can contain other Components.
 * The key insight: Composite implements Component AND contains Components.
 */
class Folder implements FileSystemComponent {
    private children: FileSystemComponent[] = [];
    
    constructor(private name: string) {}
    
    getName(): string {
        return this.name;
    }
    
    /**
     * Composite operation: delegates to children and aggregates results.
     * Client doesn't know if it's calling this on a File or Folder.
     */
    getSize(): number {
        // Recursively sum sizes of all children
        return this.children.reduce((total, child) => total + child.getSize(), 0);
    }
    
    /**
     * Composite-specific methods for managing children.
     * These don't exist in Component interface (type safety).
     */
    add(component: FileSystemComponent): void {
        this.children.push(component);
    }
    
    remove(component: FileSystemComponent): void {
        const index = this.children.indexOf(component);
        if (index > -1) {
            this.children.splice(index, 1);
        }
    }
    
    print(indent: string = ""): void {
        console.log(`${indent}📁 ${this.name} (${this.getSize()} bytes total)`);
        // Delegate printing to children with increased indentation
        this.children.forEach(child => child.print(indent + "  "));
    }
}

// Usage: Building a tree structure
const root = new Folder("root");
const documents = new Folder("documents");
const pictures = new Folder("pictures");

// Add files to folders
documents.add(new File("resume.pdf", 45000));
documents.add(new File("coverletter.docx", 12000));

pictures.add(new File("photo1.jpg", 2500000));
pictures.add(new File("photo2.jpg", 3200000));

// Nest folders
const work = new Folder("work");
work.add(new File("project.ts", 15000));
documents.add(work);

// Build tree
root.add(documents);
root.add(pictures);
root.add(new File("readme.txt", 500));

// Client code treats all uniformly
function analyzeStorage(component: FileSystemComponent): void {
    console.log(`\nAnalyzing: ${component.getName()}`);
    console.log(`Total size: ${component.getSize()} bytes`);
    component.print();
}

// Works with individual file
const singleFile = new File("note.txt", 100);
analyzeStorage(singleFile);

// Works with complex tree
analyzeStorage(root);
// Output shows recursive tree structure with sizes aggregated at each level
```

**Code Explanation**:

- **Component** (`FileSystemComponent`): Declares the interface for accessing and managing components. Crucially, `getSize()` is defined here, forcing both files and folders to implement it.
- **Leaf** (`File`): Represents end objects. Implements operations directly.
- **Composite** (`Folder`): Maintains a collection of child Components. Implements operations by delegating to children and aggregating results (summing sizes, traversing for printing).

**Transparency vs. Safety Trade-off**:
The code above puts `add()`/`remove()` only in `Folder` (type safety). An alternative puts them in the `Component` interface (transparency) but requires Leaf classes to implement no-op or throw exceptions (Liskov Substitution Principle violation). The shown approach (Component interface only for operations that make sense for both) is preferred in TypeScript/Java.

### **Connection to Other Patterns**
- **Decorator**: Both use recursive composition, but Decorator adds responsibilities to a single component, while Composite aggregates multiple components.
- **Iterator**: Often used to traverse Composite trees (Chapter 9).

---

## **7.3 Bridge Pattern: Decoupling Abstraction from Implementation**

### **Intent**
*Decouple an abstraction from its implementation so that the two can vary independently.*

### **The Problem**
Consider a shape drawing application supporting multiple shapes (Circle, Square) and multiple rendering APIs (OpenGL, DirectX, SVG). Using inheritance creates **Cartesian product explosion**:

```typescript
// ❌ INHERITANCE HELL: 2 (shapes) × 3 (APIs) = 6 classes
class CircleOpenGL extends Circle { /* OpenGL circle drawing */ }
class CircleDirectX extends Circle { /* DirectX circle drawing */ }
class CircleSVG extends Circle { /* SVG circle drawing */ }
// Repeat for Square, Triangle, etc.
```

Adding a new shape requires N new classes (one per API). Adding a new API requires M new classes (one per shape).

### **The Solution: Separate Hierarchies**

The Bridge pattern separates the **abstraction** (what the object is: Shape) from the **implementation** (how it's rendered: DrawingAPI) so they can evolve independently.

```typescript
/**
 * IMPLEMENTATION INTERFACE (The "Bridge")
 * Defines the interface for implementation classes.
 * This is the "how" - rendering details.
 */
interface DrawingAPI {
    drawCircle(x: number, y: number, radius: number): void;
    drawRectangle(x: number, y: number, width: number, height: number): void;
}

/**
 * CONCRETE IMPLEMENTATIONS
 * Different ways to render shapes.
 */
class OpenGLAPI implements DrawingAPI {
    drawCircle(x: number, y: number, radius: number): void {
        console.log(`[OpenGL] Drawing circle at (${x},${y}) r=${radius}`);
        // Actual OpenGL calls here
    }
    
    drawRectangle(x: number, y: number, width: number, height: number): void {
        console.log(`[OpenGL] Drawing rectangle at (${x},${y}) ${width}x${height}`);
    }
}

class DirectXAPI implements DrawingAPI {
    drawCircle(x: number, y: number, radius: number): void {
        console.log(`[DirectX] Rendering circle center(${x},${y}) radius=${radius}`);
    }
    
    drawRectangle(x: number, y: number, width: number, height: number): void {
        console.log(`[DirectX] Rendering rect origin(${x},${y}) dim(${width},${height})`);
    }
}

class SVGAPI implements DrawingAPI {
    drawCircle(x: number, y: number, radius: number): void {
        console.log(`<circle cx="${x}" cy="${y}" r="${radius}" />`);
    }
    
    drawRectangle(x: number, y: number, width: number, height: number): void {
        console.log(`<rect x="${x}" y="${y}" width="${width}" height="${height}" />`);
    }
}

/**
 * ABSTRACTION
 * Defines the interface for the control part of the two class hierarchies.
 * Maintains a reference to an Implementation object (the Bridge).
 */
abstract class Shape {
    // The bridge: composition over inheritance for implementation
    protected drawingAPI: DrawingAPI;
    
    constructor(drawingAPI: DrawingAPI) {
        this.drawingAPI = drawingAPI;
    }
    
    abstract draw(): void;
    abstract resize(percent: number): void;
}

/**
 * REFINED ABSTRACTIONS
 * Extend the interface defined by Abstraction.
 * These are the "what" - shape types.
 */
class CircleShape extends Shape {
    private x: number;
    private y: number;
    private radius: number;
    
    constructor(x: number, y: number, radius: number, drawingAPI: DrawingAPI) {
        super(drawingAPI);
        this.x = x;
        this.y = y;
        this.radius = radius;
    }
    
    // Delegates to implementation
    draw(): void {
        this.drawingAPI.drawCircle(this.x, this.y, this.radius);
    }
    
    resize(percent: number): void {
        this.radius *= (percent / 100);
    }
}

class RectangleShape extends Shape {
    private x: number;
    private y: number;
    private width: number;
    private height: number;
    
    constructor(x: number, y: number, w: number, h: number, drawingAPI: DrawingAPI) {
        super(drawingAPI);
        this.x = x;
        this.y = y;
        this.width = w;
        this.height = h;
    }
    
    draw(): void {
        this.drawingAPI.drawRectangle(this.x, this.y, this.width, this.height);
    }
    
    resize(percent: number): void {
        this.width *= (percent / 100);
        this.height *= (percent / 100);
    }
}

// Usage: Mix and match abstractions with implementations
const shapes: Shape[] = [
    new CircleShape(1, 2, 3, new OpenGLAPI()),
    new CircleShape(5, 7, 11, new SVGAPI()),
    new RectangleShape(0, 0, 100, 50, new DirectXAPI())
];

// Draw all shapes - each uses its own rendering API
shapes.forEach(shape => shape.draw());

// Runtime switching of implementation
const circle = new CircleShape(0, 0, 10, new OpenGLAPI());
circle.draw(); // OpenGL

// Can change implementation (Bridge allows this)
// circle = new CircleShape(0, 0, 10, new SVGAPI()); // New instance with different API
```

**Code Explanation**:

- **Abstraction** (`Shape`): Defines the high-level interface (what shapes can do: draw, resize). Maintains reference to `DrawingAPI` (the bridge).
- **Refined Abstraction** (`CircleShape`, `RectangleShape`): Concrete shape types that delegate rendering to the implementation.
- **Implementation** (`DrawingAPI`): Interface for rendering operations (the "how").
- **Concrete Implementation** (`OpenGLAPI`, `DirectXAPI`, `SVGAPI`): Specific rendering technologies.

**Benefits**:
- **Decoupling**: Shape hierarchy and DrawingAPI hierarchy evolve independently.
- **Reduced class count**: M shapes × N APIs becomes M + N classes.
- **Runtime flexibility**: Can change drawing API at runtime (though requires new instance in this example, or setter injection).

### **Bridge vs. Strategy**
Both use composition to vary behavior, but:
- **Strategy**: Client chooses algorithm for specific task (e.g., sorting strategy).
- **Bridge**: Structural separation of abstraction from implementation, usually fixed after construction but allowing both sides to vary in design.

---

## **7.4 Flyweight Pattern: Sharing State for Efficiency**

### **Intent**
*Use sharing to support large numbers of fine-grained objects efficiently.*

### **The Problem**
A word processor creates an object for every character in a document. A 10-page document with 5,000 characters would create 5,000 character objects, consuming excessive memory when most characters share the same properties (font, size, color).

### **The Solution: Intrinsic vs. Extrinsic State**

The Flyweight pattern divides object state:
- **Intrinsic state**: Shared among many objects (character code, font family). Stored in the Flyweight.
- **Extrinsic state**: Unique to each object instance (position in document, color). Passed by client or computed.

```typescript
/**
 * FLYWEIGHT INTERFACE
 * Declares methods that accept extrinsic state.
 */
interface CharacterFlyweight {
    display(extrinsicState: CharacterContext): void;
}

/**
 * CHARACTER CONTEXT (Extrinsic State)
 * Passed to flyweight methods, not stored in flyweight.
 */
interface CharacterContext {
    x: number;
    y: number;
    fontSize: number;
    color: string;
}

/**
 * CONCRETE FLYWEIGHT
 * Stores intrinsic state (shared data).
 * In a real app, this would be much larger (glyph paths, font metrics).
 */
class ConcreteCharacter implements CharacterFlyweight {
    // INTRINSIC STATE: Shared across all instances of this character
    private char: string;
    private fontFamily: string;
    private glyphPath: string; // Complex path data (expensive)
    
    constructor(char: string, fontFamily: string) {
        this.char = char;
        this.fontFamily = fontFamily;
        this.glyphPath = this.loadGlyphPath(char, fontFamily);
        console.log(`Creating flyweight for '${char}' in ${fontFamily}`);
    }
    
    private loadGlyphPath(char: string, font: string): string {
        // Simulate expensive operation loading vector paths
        return `vector-path-${char}-${font}`;
    }
    
    // EXTRINSIC STATE passed as parameter (position, size, color)
    display(context: CharacterContext): void {
        console.log(
            `Rendering '${this.char}' at (${context.x},${context.y}) ` +
            `size=${context.fontSize} color=${context.color} ` +
            `using path=${this.glyphPath}`
        );
    }
}

/**
 * FLYWEIGHT FACTORY
 * Creates and manages flyweight objects, ensuring proper sharing.
 * When client requests a flyweight, factory supplies existing instance
 * or creates new one if it doesn't exist.
 */
class CharacterFactory {
    private flyweights: Map<string, ConcreteCharacter> = new Map();
    
    getCharacter(char: string, fontFamily: string): CharacterFlyweight {
        const key = `${char}-${fontFamily}`;
        
        if (!this.flyweights.has(key)) {
            this.flyweights.set(key, new ConcreteCharacter(char, fontFamily));
        }
        
        return this.flyweights.get(key)!;
    }
    
    getFlyweightCount(): number {
        return this.flyweights.size;
    }
}

/**
 * CLIENT
 * Maintains extrinsic state and passes it to flyweight.
 */
class Document {
    private factory: CharacterFactory;
    // Stores only references to flyweights + extrinsic state
    private characters: Array<{ flyweight: CharacterFlyweight, context: CharacterContext }> = [];
    
    constructor() {
        this.factory = new CharacterFactory();
    }
    
    addCharacter(char: string, font: string, x: number, y: number, size: number, color: string): void {
        const flyweight = this.factory.getCharacter(char, font);
        const context: CharacterContext = { x, y, fontSize: size, color };
        this.characters.push({ flyweight, context });
    }
    
    render(): void {
        this.characters.forEach(({ flyweight, context }) => {
            flyweight.display(context);
        });
    }
    
    getMemoryStats(): void {
        console.log(`\nTotal characters in document: ${this.characters.length}`);
        console.log(`Unique flyweights created: ${this.factory.getFlyweightCount()}`);
        console.log(`Memory saved: ${this.characters.length - this.factory.getFlyweightCount()} objects`);
    }
}

// Usage
const doc = new Document();

// Adding many characters - only unique (char, font) pairs create flyweights
doc.addCharacter('H', 'Arial', 0, 0, 12, 'black');
doc.addCharacter('e', 'Arial', 10, 0, 12, 'black');
doc.addCharacter('l', 'Arial', 20, 0, 12, 'black');
doc.addCharacter('l', 'Arial', 30, 0, 12, 'black'); // Reuses 'l' flyweight
doc.addCharacter('o', 'Arial', 40, 0, 12, 'black');
doc.addCharacter('H', 'Arial', 50, 0, 14, 'red'); // Reuses 'H' flyweight (font same)
doc.addCharacter('H', 'Times', 0, 20, 12, 'black'); // New flyweight (different font)

doc.render();
doc.getMemoryStats();
// Output shows 6 characters but only 6 unique flyweights (not 7, 'l' and 'H' reused)
```

**Code Explanation**:

- **Flyweight** (`CharacterFlyweight`): Interface accepting extrinsic state.
- **ConcreteFlyweight** (`ConcreteCharacter`): Stores intrinsic state (char, font, glyphPath). This is the shared object—expensive to create, so we minimize instances.
- **FlyweightFactory** (`CharacterFactory`): Manages sharing. Ensures that for each unique intrinsic state combination, only one flyweight exists.
- **Client** (`Document`): Maintains the collection of flyweights and their extrinsic state (context). When rendering, passes extrinsic state to the shared flyweight.

**Memory Impact**:
- Without Flyweight: 10,000 characters × 1KB each = 10MB.
- With Flyweight: 100 unique glyphs × 1KB + 10,000 contexts × 20 bytes = 100KB + 200KB = 300KB (97% reduction).

### **Thread Safety Considerations**
Flyweights are often shared across threads. If they contain any mutable state (they shouldn't), synchronization is required. Best practice: **Flyweights must be immutable**.

### **When NOT to Use Flyweight**
- When object count is small (overhead not justified).
- When extrinsic state computation is more expensive than storing intrinsic state.
- When identity matters (Flyweights sacrifice object identity—two "A" characters share the same instance).

---

## **Chapter Summary**
In this chapter, we explored four structural composition patterns:

1.  **Decorator**: Wraps objects to add responsibilities dynamically. Uses recursive composition to stack behaviors. Alternative to subclassing when extensions are needed at runtime. **Status: Essential for I/O streams and UI enhancements.**

2.  **Composite**: Treats individual objects and compositions uniformly through a common interface. Enables tree structures and part-whole hierarchies (file systems, UI trees). **Status: Fundamental for hierarchical data structures.**

3.  **Bridge**: Decouples abstraction from implementation, allowing both to vary independently. Prevents Cartesian product class explosion. **Status: Critical for cross-platform and driver architectures.**

4.  **Flyweight**: Shares intrinsic state among many objects to minimize memory usage. Separates shared and unique state. **Status: Specialized for large-scale object systems (games, text editors).**

**Key Distinctions**:
- **Decorator** adds responsibilities to one object.
- **Composite** aggregates many objects into tree structures.
- **Bridge** separates abstraction from implementation hierarchies.
- **Flyweight** shares objects to reduce memory.

---

## **Next Chapter Preview**
**Chapter 8: Algorithms and Responsibilities (Strategy, Template Method, Command)**

We transition from Structural to **Behavioral** patterns—concerned with algorithms and the assignment of responsibilities between objects. Chapter 8 begins with the **Strategy** pattern for interchangeable algorithms, the **Template Method** for algorithm skeletons with variant steps, and the **Command** pattern for encapsulating requests as objects (enabling undo, queuing, and macro recording). These patterns address the fundamental challenge of making algorithms flexible, reusable, and decoupled from the objects that use them.

---



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='6. adapters_and_wrappers.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='8. algorithms_and_responsibilities.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
